In [1]:
from pytket import Circuit, OpType
from pytket.predicates import GateSetPredicate, NoMidMeasurePredicate, ConnectivityPredicate

circ = Circuit(2, 2)
circ.Rx(0.2, 0).CX(0, 1).Rz(-0.7, 1).measure_all()

gateset = GateSetPredicate({OpType.Rx, OpType.CX, OpType.Rz, OpType.Measure})
midmeasure = NoMidMeasurePredicate()

print(gateset.verify(circ))
print(midmeasure.verify(circ))

circ.S(0)

print(gateset.verify(circ))
print(midmeasure.verify(circ))

True
True
False
False


In [2]:
from pytket.passes import RebaseTket, RebaseCustom

In [48]:
from pytket import Circuit, OpType
from pytket.passes import RebaseCustom, AutoRebase

gates = {OpType.Rz, OpType.Ry, OpType.CY, OpType.ZZPhase}
cx_in_cy = Circuit(2)
cx_in_cy.Rz(0.5, 1).CY(0, 1).Rz(-0.5, 1)

def tk1_to_rzry(a, b, c):
    circ = Circuit(1)
    circ.Rz(c + 0.5, 0).Ry(b, 0).Rz(a - 0.5, 0)
    return circ

custom = RebaseCustom(gates, cx_in_cy, tk1_to_rzry)

circ = Circuit(3)
circ.X(0).CX(0, 1).Ry(0.2, 1)
circ.ZZPhase(-0.83, 2, 1).Rx(0.6, 2)

custom.apply(circ)

print(circ.get_commands())

[Rz(0.5) q[0];, Rz(0.5) q[1];, Ry(1) q[0];, Rz(3.5) q[0];, CY q[0], q[1];, Rz(3.5) q[1];, Ry(0.2) q[1];, ZZPhase(3.17) q[2], q[1];, Rz(0.5) q[2];, Ry(0.6) q[2];, Rz(3.5) q[2];]


In [49]:
from pytket import passes

passes.AutoRebase({OpType.U3, OpType.CX}).apply(circ)
passes.AutoSquash({OpType.U3}).apply(circ)
# passes.FullPeepholeOptimise(allow_swaps=False).apply(circ)
# AutoRebase({OpType.U3, OpType.CX}).apply(circ)


True

In [50]:
render_circuit_jupyter(circ)

In [6]:
from pytket.circuit import Unitary2qBox

In [10]:
import cirq

sqrt_iswap = Unitary2qBox(cirq.unitary(cirq.SQRT_ISWAP))

In [ ]:
from regulus import gates

gates.SQiSW

In [28]:
sqrt_iswap.type

OpType.Unitary2qBox

In [16]:
from pytket.circuit.display import render_circuit_jupyter


render_circuit_jupyter(sqrt_iswap.get_circuit())

In [25]:
from pytket.passes import FullPeepholeOptimise
成 = sqrt_iswap.get_circuit()
FullPeepholeOptimise(target_2qb_gate=OpType.TK2).apply(成)

True

In [26]:
render_circuit_jupyter(成)

In [ ]:
from pytket import Circuit
from pytket.extensions.qiskit import IBMQBackend
from pytket.passes import PlacementPass, RoutingPass
from pytket.predicates import ConnectivityPredicate
from pytket.placement import GraphPlacement


circ = Circuit(4, 4)
circ.H(0).H(1).H(2).V(3)
circ.CX(0, 1).CX(1, 2).CX(2, 3)
circ.Rz(-0.37, 3)
circ.CX(2, 3).CX(1, 2).CX(0, 1)
circ.H(0).H(1).H(2).Vdg(3)
circ.measure_all()

backend = IBMQBackend("ibmq_quito")
place = PlacementPass(GraphPlacement(backend.backend_info.architecture))
place.apply(circ)

print(circ.get_commands())
print(ConnectivityPredicate(backend.backend_info.architecture).verify(circ))

AccountNotFoundError: 'Unable to find account.'

In [52]:
from pytket import Circuit, OpType
from pytket.passes import RemoveRedundancies

circ = Circuit(3, 3)
circ.Rx(0.92, 0).CX(1, 2).Rx(-0.18, 0)  # Adjacent Rx gates can be merged
circ.CZ(0, 1).Ry(0.11, 2).CZ(0, 1)      # CZ is self-inverse
circ.XXPhase(0.6, 0, 1)
circ.YYPhase(0, 0, 1)    # 0-angle rotation does nothing
circ.ZZPhase(-0.84, 0, 1)
circ.Rx(0.03, 0).Rz(-0.9, 1).measure_all()  # Effect of Rz is eliminated by measurement

render_circuit_jupyter(circ)

RemoveRedundancies().apply(circ)
print(circ.get_commands())

[Rx(0.74) q[0];, CX q[1], q[2];, XXPhase(0.6) q[0], q[1];, Ry(0.11) q[2];, Measure q[2] --> c[2];, ZZPhase(3.16) q[0], q[1];, Measure q[1] --> c[1];, Rx(0.03) q[0];, Measure q[0] --> c[0];]


In [53]:
render_circuit_jupyter(circ)

In [ ]:
from pytket.circuit import Circuit, PauliExpBox
from pytket.passes import PauliSimp, DecomposeBoxes
from pytket.pauli import Pauli
from pytket.transform import CXConfigType
from pytket.utils import Graph

pauli_XYXZYXZZ = PauliExpBox([Pauli.X, Pauli.Y, Pauli.X, Pauli.Z, Pauli.Y, Pauli.X, Pauli.Z, Pauli.Z], 0.42)

circ = Circuit(8)
circ.add_gate(pauli_XYXZYXZZ, [0, 1, 2, 3, 4, 5, 6, 7])

PauliSimp(cx_config=CXConfigType.Tree).apply(circ)
# DecomposeBoxes().apply(circ)
print(circ.get_commands())
render_circuit_jupyter(circ)

[TK1(0.5, 0.5, 0.5) q[0];, TK1(0, 1.5, 0) q[1];, TK1(0.5, 0.5, 0.5) q[2];, TK1(0, 1.5, 0) q[4];, TK1(0, 1.5, 3.5) q[5];, CX q[7], q[6];, CX q[1], q[0];, CX q[3], q[2];, CX q[5], q[4];, CX q[2], q[0];, CX q[6], q[4];, CX q[4], q[0];, TK1(0, 0, 2.42) q[0];, CX q[4], q[0];, CX q[2], q[0];, TK1(0, 0.5, 0) q[4];, CX q[1], q[0];, CX q[3], q[2];, CX q[6], q[4];, TK1(0.5, 0.5, 0.5) q[0];, TK1(0, 0.5, 0) q[1];, TK1(0.5, 0.5, 0.5) q[2];, CX q[5], q[4];, CX q[7], q[6];, TK1(0.5, 0.5, 0) q[5];]


In [75]:
circ = Circuit(8)
circ.add_pauliexpbox( PauliExpBox([Pauli.X, Pauli.Y, Pauli.X, Pauli.Z, Pauli.Y, Pauli.X, Pauli.Z, Pauli.Z], 0.42), [0, 1, 2, 3, 4, 5, 6, 7])
passes.DecomposeBoxes().apply(circ)

workflow = passes.RepeatWithMetricPass(passes.PauliSimp(), lambda circ: circ.n_2qb_gates())
workflow.apply(circ)

render_circuit_jupyter(circ)

RuntimeError: Cannot compose these Compiler Passes due to mismatching Predicates of type: GateSetPredicate

In [68]:
passes.SequencePass([passes.PauliSimp(), passes.FullPeepholeOptimise()]).apply(circ)

True

In [77]:
passes.SynthesiseTK().apply(circ)

render_circuit_jupyter(circ)